In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [2]:
HOST = "datasetint.dev.verta.ai"

In [3]:
import os
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

In [4]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [5]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

In [6]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)

set email from environment
set developer key from environment
connection successfully established


In [9]:
dataset1 = client.set_dataset(name="Demo Dataset S3", type="s3")
print("Dataset 1: {}".format(dataset1))

dataset2 = client.set_dataset(name="Demo Dataset S3 - 2", type="s3", tags=["tag-1","tag-2","tag-3"], desc="S3 Dataset with tags and description")
print("Dataset 2: {}".format(dataset2))

version = dataset1.create_version(bucket_name="verta-starter")

set existing Dataset: Demo Dataset S3 from personal workspace
Dataset 1: id: "12"
name: "Demo Dataset S3"
owner: "1003"
time_created: 1593574504464
time_updated: 1593576518537
workspace_id: "1003"
workspace_type: USER

set existing Dataset: Demo Dataset S3 - 2 from personal workspace
Dataset 2: id: "13"
name: "Demo Dataset S3 - 2"
owner: "1003"
description: "S3 Dataset with tags and description"
tags: "tag-1"
tags: "tag-2"
tags: "tag-3"
time_created: 1593574505397
time_updated: 1593574505405
workspace_id: "1003"
workspace_type: USER

created new DatasetVersion: 0


In [10]:
version = dataset1.create_version(bucket_name="verta-starter")

created new DatasetVersion: 0


In [20]:
version.id

'bd0332ba2d95e4ccc614978df2136fbd33961352b326690977fa680ffe544e45'

In [12]:
version

location_type: S3_FILE_SYSTEM
size: 370364312
dataset_part_infos {
  path: "spam.csv"
  size: 503663
  checksum: "\"008dcf9ca6aab621d3b3831db6815867\""
  last_modified_at_source: 1574898082000
}
dataset_part_infos {
  path: "census-cleaned.csv"
  size: 9950413
  checksum: "\"7b0b17a5271852616ed19a72914e776d\""
  last_modified_at_source: 1590635928000
}
dataset_part_infos {
  path: "english-tweets.csv"
  size: 124090700
  checksum: "\"15da0ba7835922e239a43a6ec2a8356c-8\""
  last_modified_at_source: 1584653133000
}
dataset_part_infos {
  path: "reviews.ann"
  size: 22029640
  checksum: "\"f5887481b609b7dba1974a9ef9deac79-3\""
  last_modified_at_source: 1574904181000
}
dataset_part_infos {
  path: "positive-english-tweets.csv"
  size: 61900256
  checksum: "\"9a1c0f0d272bfb2db20fe5791e6483d3-4\""
  last_modified_at_source: 1584653133000
}
dataset_part_infos {
  path: "german-tweets.csv"
  size: 3197125
  checksum: "\"afcc5b2bc3b206f786c45e72b87e44fa\""
  last_modified_at_source: 1584653133

In [32]:
client._conn.auth

{'Grpc-Metadata-source': 'PythonClient',
 'Grpc-Metadata-email': 'ravi@verta.ai',
 'Grpc-Metadata-developer_key': '10c7618d-8120-451d-bc02-b39ab0668cf5',
 'Grpc-Metadata-scheme': 'https'}

In [57]:
url = "{}://{}/v1/dataset-version/findDatasetVersions".format(
            client._conn.scheme,
            client._conn.socket,
        )
#Message = client._DatasetVersionService.FindDatasetVersions
msg = {'dataset_id':dataset1.id, 'dataset_version_ids':[version.id]}
#payload = {'key1': 'value1', 'key2': 'value2'}
msg = {'dataset_id': ''}
import requests
r = requests.post(url, data = msg, headers= client._conn.auth)
#r = requests.Request('POST', url, data = msg, headers= client._conn.auth)
r.text

'{"error":"invalid character \'d\' looking for beginning of value","code":3,"message":"invalid character \'d\' looking for beginning of value"}'

In [51]:
verta._utils.make_request()

AttributeError: module 'verta' has no attribute '_utils'

In [ ]:
find_dataset_output = client.find_datasets(name=dataset1["name"], tags=["tag-1"])
print("Find Datasets output: {}".format(find_dataset_output))